<a href="https://colab.research.google.com/github/igorcoa/TecControleQualidade/blob/main/%C3%8DndicesDeCapacidadeDeProcesso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Bibliotecas

In [ ]:
!pip install -U kora

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 


In [ ]:
import numpy as np
from numpy.random import normal
import math
import statistics as st
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import figure
import glob
from PIL import Image
from google.colab import files
import imageio
import moviepy.editor as mp
from kora.drive import upload_public
import time
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go

from google.colab import data_table
data_table.enable_dataframe_formatter()

def dfprint(df, include_index=False,linhas=10):
  return data_table.DataTable(df, include_index=include_index,num_rows_per_page=linhas)

def dict_mean(d):
  soma = 0
  for val in d.values():
    soma += val
  return (soma/len(d))

from IPython.display import HTML, display
from IPython.display import clear_output
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))
out = display(progress(0, 5000), display_id=True)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2400256/45929032 bytes (5.2%)5636096/45929032 bytes (12.3%)8724480/45929032 bytes (19.0%)12279808/45929032 bytes (26.7%)15605760/45929032 bytes (34.0%)18907136/45929032 bytes (41.2%)22142976/45929032 bytes (48.2%)24387584/45929032 bytes (53.1%)27156480/45929032 bytes (59.1%)29679616/45929032 bytes (64.6%)32759808/45929032 bytes (71.3%)36192256/45929032 bytes (78.8%)

# Avaliação do Sistema de Medição

## Construindo a Função

Considerando o exemplo proposto, onde somos responsáveis pela medição de placas de metal em uma indústria metalúrgica, precisamos de uma função que faça a avaliação deste sistema de medição. Para a didática do problema, é útil, também, que nossa função simule a tomada de medidas.

Começamos, então, montando uma tabela contendo $n$ peças amostradas por 3 operadores que realizam 3 medidas diferentes em cada peça. Na prática, os operadores não saberiam se estão medindo a mesma peça, cabendo ao gestor entregar as peças para medição e anotar devidamente os valores na tabela.

Resumo do Problema:
<center><font size = 5>Placas de $3mm$</font></center>
<center><font size = 5>$n = 50$ peças selecionadas aleatoriamente</font></center>
<center><font size = 5>3 operadores medem três vezes cada peça</font></center>

In [ ]:
medidas_nomesColuna = ["Peça","Op1_Med1","Op1_Med2","Op1_Med3",
                       "Op2_Med1","Op2_Med2","Op2_Med3",
                       "Op3_Med1","Op3_Med2","Op3_Med3",]
medidas=[]
n=50 #número de peças amostradas
dp = 0.1 #desvio padrão das medidas
for p in range(n):
  peca = [p+1]
  pecaReal = normal(loc=3,scale=0.1)
  medidor1 = np.round(pecaReal + normal(size=3,scale=dp),3).tolist()
  medidor2 = np.round(pecaReal + normal(size=3,scale=dp),3).tolist()
  medidor3 = np.round(pecaReal + normal(size=3,scale=dp),3).tolist()
  peca.extend(medidor1)
  peca.extend(medidor2)
  peca.extend(medidor3)
  medidas.append(peca)
medidas = pd.DataFrame(medidas,columns=medidas_nomesColuna)
dfprint(medidas)

,Peça,Op1_Med1,Op1_Med2,Op1_Med3,Op2_Med1,Op2_Med2,Op2_Med3,Op3_Med1,Op3_Med2,Op3_Med3
0,1,2.904,2.944,2.878,2.855,2.821,2.846,3.003,2.856,2.914
1,2,2.990,2.952,3.279,2.991,2.831,3.127,3.118,2.920,2.850
2,3,3.310,3.075,3.179,3.042,3.045,2.999,3.104,3.209,3.079
3,4,3.023,2.847,2.964,3.002,3.064,3.164,3.073,3.206,3.087
4,5,3.076,3.120,3.098,3.086,3.116,3.186,2.996,3.179,3.191
5,6,3.204,3.228,3.201,3.322,3.235,3.119,3.087,3.026,3.247
6,7,2.965,3.045,3.149,2.922,3.026,3.136,3.076,3.217,2.897
7,8,3.156,2.827,2.799,2.896,2.678,2.778,2.774,2.913,2.900
8,9,3.031,2.884,2.964,3.008,3.023,3.133,3.027,2.994,3.130
9,10,3.089,2.988,3.059,3.142,3.075,3.098,3.046,3.064,3.215


Agora é importante montarmos uma nova tabela, contendo o $\bar{X}$ e o $R$ de cada operador (média e amplitude amostrais, respectivamente). Podemos fazer isso através de:

In [ ]:
mediaAmplitude = []
mediaAmplitude_nomesColuna = ["Peça","Op1_xBarra","Op1_R",
                              "Op2_xBarra","Op2_R",
                              "Op3_xBarra","Op3_R",
                              "xBarra","R"]
pecaNum = 1
for i in medidas.iterrows():
  p = i[1]
  peca = [pecaNum]
  op1 = [p.Op1_Med1,p.Op1_Med2,p.Op1_Med3]
  op2 = [p.Op2_Med1,p.Op2_Med2,p.Op2_Med3]
  op3 = [p.Op3_Med1,p.Op3_Med2,p.Op3_Med3]
  op = []
  op.extend(op1)
  op.extend(op2)
  op.extend(op3)
  op1_xBarra = round(np.mean(op1),3)
  op1_R = round(max(op1)-min(op1),3)
  op2_xBarra = round(np.mean(op2),3)
  op2_R = round(max(op2)-min(op2),3)
  op3_xBarra = round(np.mean(op3),3)
  op3_R = round(max(op3)-min(op3),3)
  xB = round(np.mean(op),3)
  R = round(max(op)-min(op),3)
  peca.append(op1_xBarra)
  peca.append(op1_R)
  peca.append(op2_xBarra)
  peca.append(op2_R)
  peca.append(op3_xBarra)
  peca.append(op3_R)
  peca.append(xB)
  peca.append(R)
  mediaAmplitude.append(peca)
  pecaNum+=1
mediaAmplitude = pd.DataFrame(mediaAmplitude,columns=mediaAmplitude_nomesColuna)
dfprint(mediaAmplitude)

,Peça,Op1_xBarra,Op1_R,Op2_xBarra,Op2_R,Op3_xBarra,Op3_R,xBarra,R
0,1,2.986,0.077,2.852,0.107,2.922,0.368,2.920,0.368
1,2,3.002,0.337,2.917,0.017,3.005,0.174,2.975,0.337
2,3,2.881,0.133,2.795,0.299,2.805,0.239,2.827,0.307
3,4,3.052,0.153,3.098,0.082,3.135,0.096,3.095,0.191
4,5,3.049,0.331,3.036,0.057,2.923,0.119,3.003,0.365
5,6,2.961,0.150,3.077,0.263,3.049,0.151,3.029,0.322
6,7,2.958,0.166,2.983,0.135,2.983,0.129,2.975,0.166
7,8,3.121,0.106,3.084,0.337,3.096,0.046,3.100,0.337
8,9,3.084,0.134,3.075,0.062,2.960,0.185,3.040,0.271
9,10,3.140,0.110,3.102,0.147,3.027,0.079,3.090,0.224


Finalmente, podemos entender entender exatamente ao que se refere cada erro. Para isso, uma função toma as médias de cada $\bar{X}$, chamado de $\bar{\bar{X}}$; além disso, a função também toma as médias das amplitudes, o $\bar{R}$. Calculamos então a repetibilidade, que diz respeito à comparação entre os operadores consigo mesmo, ou seja, compara-se o operador 1 com ele mesmo, o 2 com ele mesmo e o 3 com ele mesmo, por fim comparando as 3 comparações entre si. Isso é obtido através do:
<br><br>
<center><font size = 5>$\hat{\sigma}_{repe}=\frac{\bar{R}}{d_2}$</font></center>

<br>
onde o $d_2$ é tabelado para o número de operadores e/ou medições por cada operador.
<br>

Depois podemos obter a reprodutibilidade, que compara a reprodução da mesma medida de cada peça; fazemos isso com a fórmula:
<br><br>
<center><font size = 5>$\hat{\sigma}_{repro}=\sqrt{\left(\frac{R_{\bar{X}}}{d_2}\right)^2-\frac{\left(\hat{\sigma}_{repe}\right)^2}{n \: \cdot \: r}}$</font></center>

<br>
onde o $n$ é o número de amostras totais e $r$ é o número de operadores.
<br>

Por fim, combinamos ambos os valores no índice R&R, que é igual a 6 vezes a distância euclidiana entre a repetibilidade e a reprodutibilidade:
<br><br>
<center><font size = 5>$R\&R\:=6\cdot \sqrt{\left(\hat{\sigma}_{repro}\right)^2+\left(\hat{\sigma}_{repe}\right)^2}$</font></center>
<br>

Dipospomos dodos estes desvios na largura de 3 desvios padrões para cima e 3 para baixo. Além disso, é útil entender, também, o porcentagem que cada tipo de erro tem no Índice R&R final. Tudo isso é feito através das linhas:


In [ ]:
x2barras = {"op1" : round(np.mean(mediaAmplitude.Op1_xBarra),5),
            "op2" : round(np.mean(mediaAmplitude.Op2_xBarra),5),
            "op3" : round(np.mean(mediaAmplitude.Op3_xBarra),5)}
rBarra = {"op1" : round(np.mean(mediaAmplitude.Op1_R),5),
          "op2" : round(np.mean(mediaAmplitude.Op2_R),5),
          "op3" : round(np.mean(mediaAmplitude.Op3_R),5)}
r2barras = dict_mean(rBarra)
d_2 = 1.693
delta_repe = r2barras/d_2
x2barras_amplitude = max(x2barras.values())-min(x2barras.values())
try:
  delta_repro = math.sqrt(((x2barras_amplitude/d_2)**2)-((delta_repe**2)/(n*3)))
except:
  delta_repro=0
ReR = 6*math.sqrt((delta_repro**2)+(delta_repe**2))
print("                    Desvio    %")
print("Reprodutibilidade ",round(delta_repro*6,5),round(delta_repro*100*6/ReR,5))
print("Repetibilidade    ",round(delta_repe*6,5),round(delta_repe*100*6/ReR,5))
print("Índice R$R        ",round(ReR,5))

                    Desvio    %
Reprodutibilidade  0.06434 10.83839
Repetibilidade     0.5901  99.41091
Índice R$R         0.5936


Ou seja, para uma placa de $3mm$, a largura de R&R contém $99,73\%$ dos dados. Isto é, nossa chapa de metal teria $3\pm \frac{R\&R}{2}mm$.

Vejamos agora a capacidade individual de cada medidor, através da fórmula:
<br><br>
<center><font size = 5>$\hat{\sigma}_{operador\:i}=\frac{\bar{R}_i}{d_2}$</font></center>


In [ ]:
delta_medidor = {"op1" : 0,
                 "op2" : 0,
                 "op3" : 0}
for rB in rBarra:
  delta_medidor[rB] = rBarra[rB]*6/d_2

delta_medidor

{'op1': 0.6155227406969875,
 'op2': 0.5421618428824572,
 'op3': 0.6126166568222091}

Podemos agora calcular o desvio padrão peça a peça, que se refere a variabilidade do processo. Com ela e o R&R, conseguimos calcular o desvio padrão total, que é a distância euclidiana entre ambos os desvios.

A função a seguir calcula isso e também a porcentagem de influência de cada desvio no desvio total final.

In [ ]:
pecaPeca = round(st.stdev(mediaAmplitude.xBarra),5)
desvPtotal = math.sqrt((pecaPeca**2)+(ReR/6)**2)
print("                           Desvio   %")
print("Índice R&R               ",round(ReR,5),
      round(100*ReR/6/desvPtotal,2))
print("Desvio Padrão Peça a Peça",
      round(6*pecaPeca,5),
      round(100*pecaPeca/desvPtotal,2))
print("Desvio Padrão Total      ",round(6*desvPtotal,5))

                           Desvio   %
Índice R&R                0.5936  68.76
Desvio Padrão Peça a Peça 0.62676 72.61
Desvio Padrão Total       0.86324


Por fim, podemos fornecer os Limites de Especificação (ou Tolerância) e assim calcular a capacidade do processo $C_p$ e a capacidade unilateral do processo $C_{pk}$. Estas capacidades são dadas pelas fórmulas:
<br><br>
<center><font size = 5>$C_p=\frac{LSE-LIE}{R\&R}$</font></center>
<br><br>
<center><font size = 5>$C_{pk}=min\{\frac{LSE-\bar{\bar{\bar{x}}}}{2\:\cdot\: R\&R},\frac{\bar{\bar{\bar{x}}}-LIE}{2\:\cdot\: R\&R}\}$</font></center>

Obtemos eles pelo código:



In [ ]:
limSupEsp = 3.1
limInfEsp = 2.7
x3barras = np.mean(mediaAmplitude.xBarra)
cp = (limSupEsp-limInfEsp)/(ReR)
cpk = min((limSupEsp-x3barras)/(ReR/2),
          (x3barras-limInfEsp)/(ReR/2))
print("Cp ",cp)
print("Cpk",cpk)

Cp  0.673857594720972
Cpk 0.3378721979930944


## Função Pronta

Agora podemos unir todos os códigos vistos até aqui em uma única função, que retorne por fim os r

In [ ]:
#compilando numa única função

def dict_mean(d):
  soma = 0
  for val in d.values():
    soma += val
  return (soma/len(d))

def avSisMed(limSupEsp = 0,limInfEsp = 0,dp=0.05,n=50,m1=0,m2=0,m3=0,dp1=0,dp2=0,dp3=0,dpPeca=0.25):
    medidas_nomesColuna = ["Peça","Op1_Med1","Op1_Med2","Op1_Med3",
                           "Op2_Med1","Op2_Med2","Op2_Med3",
                           "Op3_Med1","Op3_Med2","Op3_Med3",]
    medidas=[]
    for p in range(n):
      peca = [p+1]
      pecaReal = normal(loc=3,scale=dpPeca)
      medidor1 = np.round(pecaReal + m1 + normal(size=3,scale=dp+dp1),3).tolist()
      medidor2 = np.round(pecaReal + m2 + normal(size=3,scale=dp+dp2),3).tolist()
      medidor3 = np.round(pecaReal + m3 + normal(size=3,scale=dp+dp3),3).tolist()
      peca.extend(medidor1)
      peca.extend(medidor2)
      peca.extend(medidor3)
      medidas.append(peca)
    medidas = pd.DataFrame(medidas,columns=medidas_nomesColuna)   
    mediaAmplitude = []
    mediaAmplitude_nomesColuna = ["Peça","Op1_xBarra","Op1_R",
                                  "Op2_xBarra","Op2_R",
                                  "Op3_xBarra","Op3_R",
                                  "xBarra","R"]
    pecaNum = 1
    for i in medidas.iterrows():
      p = i[1]
      peca = [pecaNum]
      op1 = [p.Op1_Med1,p.Op1_Med2,p.Op1_Med3]
      op2 = [p.Op2_Med1,p.Op2_Med2,p.Op2_Med3]
      op3 = [p.Op3_Med1,p.Op3_Med2,p.Op3_Med3]
      op = []
      op.extend(op1)
      op.extend(op2)
      op.extend(op3)
      op1_xBarra = round(np.mean(op1),3)
      op1_R = round(max(op1)-min(op1),3)
      op2_xBarra = round(np.mean(op2),3)
      op2_R = round(max(op2)-min(op2),3)
      op3_xBarra = round(np.mean(op3),3)
      op3_R = round(max(op3)-min(op3),3)
      xB = round(np.mean(op),3)
      R = round(max(op)-min(op),3)
      peca.append(op1_xBarra)
      peca.append(op1_R)
      peca.append(op2_xBarra)
      peca.append(op2_R)
      peca.append(op3_xBarra)
      peca.append(op3_R)
      peca.append(xB)
      peca.append(R)
      mediaAmplitude.append(peca)
      pecaNum+=1
    mediaAmplitude = pd.DataFrame(mediaAmplitude,columns=mediaAmplitude_nomesColuna)   
    x2barras = {"op1" : round(np.mean(mediaAmplitude.Op1_xBarra),5),
                "op2" : round(np.mean(mediaAmplitude.Op2_xBarra),5),
                "op3" : round(np.mean(mediaAmplitude.Op3_xBarra),5)}
    rBarra = {"op1" : round(np.mean(mediaAmplitude.Op1_R),5),
              "op2" : round(np.mean(mediaAmplitude.Op2_R),5),
              "op3" : round(np.mean(mediaAmplitude.Op3_R),5)}
    r2barras = dict_mean(rBarra)
    d_2 = 1.693
    delta_repe = (r2barras)/d_2
    x2barras_amplitude = max(x2barras.values())-min(x2barras.values())
    try:
      delta_repro = math.sqrt(((x2barras_amplitude/d_2)**2)-((delta_repe**2)/(3*n)))
    except:
      delta_repro=0
    ReR = 6*math.sqrt((delta_repro**2)+(delta_repe**2))    
    delta_medidor = {"op1" : 0,
                     "op2" : 0,
                     "op3" : 0}
    for rB in rBarra:
      delta_medidor[rB] = round(rBarra[rB]*6/d_2,3)   
    todasMedidas = []
    for c in medidas:
      if c!="Peça":
        todasMedidas.extend(medidas[c].tolist())
    x3barras = np.mean(todasMedidas)
    sigma = 0
    for x in todasMedidas:
      sigma += ((x-x3barras)**2)
    delta_total = (math.sqrt(sigma))/(len(todasMedidas))
    porCent_ReR = ReR/delta_total/6*100

    pecaPeca = round(st.stdev(mediaAmplitude.xBarra),5)
    
    cp = (limSupEsp-limInfEsp)/(ReR)
    cpk = min((limSupEsp-x3barras)/(ReR/2),(x3barras-limInfEsp)/(ReR/2))

    if cpk<1:
      if (limSupEsp-x3barras)/(ReR/2)==min((limSupEsp-x3barras)/(ReR/2),(x3barras-limInfEsp)/(ReR/2)):
        falha = "Superior"
      else:
        falha = "Inferior"
    else:
      falha = "Não há"

    desvPtotal = math.sqrt((pecaPeca**2)+(ReR/6)**2)

    if limSupEsp == 0 or limInfEsp == 0:
      d = {"R&R" : round(ReR,3),
          "Capacidade dos Medidores" : delta_medidor,
          # "Capacidade do Processo (Cp)" : round(cp,5),
          # "Capabilidade do Processo (Cpk)" : round(cpk,5),
          # "Falha na Capabilidade" : falha,
          #  "Limites Especificados" : [limInfEsp,limSupEsp],
          "Média Amostral" : round(x3barras,5),
          "Desvio Padrão Peça a Peça" : round(pecaPeca*6,3),
          "Desvio Padrão Total" : round(desvPtotal*6,3),
          "%R&R" : round(100*ReR/6/desvPtotal,2),
          "%Peça a Peça" : round(100*pecaPeca/desvPtotal,2),
          "Reprodutibilidade" : round(delta_repro,5),
          "Repetibilidade" : round(delta_repe,5)}
    else:
      d = {"R&R" : round(ReR,3),
          "Capacidade dos Medidores" : delta_medidor,
          "Capacidade do Processo (Cp)" : round(cp,5),
          "Capabilidade do Processo (Cpk)" : round(cpk,5),
          "Falha na Capabilidade" : falha,
          "Limites Especificados" : [limInfEsp,limSupEsp],
          "Média Amostral" : round(x3barras,5),
          "Desvio Padrão Peça a Peça" : round(pecaPeca*6,3),
          "Desvio Padrão Total" : round(desvPtotal*6,3),
          "%R&R" : round(100*ReR/6/desvPtotal,2),
          "%Peça a Peça" : round(100*pecaPeca/desvPtotal,2),
          "Reprodutibilidade" : round(delta_repro,5),
          "Repetibilidade" : round(delta_repe,5)}
    return d,medidas,mediaAmplitude

In [ ]:
avSisMed(n=200,dp=0.01,dpPeca=0.1,dp1=0.005,limSupEsp=3.1,limInfEsp=2.7)[0]

{'%Peça a Peça': 99.37,
 '%R&R': 11.23,
 'Capabilidade do Processo (Cpk)': 3.29505,
 'Capacidade do Processo (Cp)': 5.76439,
 'Capacidade dos Medidores': {'op1': 0.088, 'op2': 0.06, 'op3': 0.06},
 'Desvio Padrão Peça a Peça': 0.614,
 'Desvio Padrão Total': 0.618,
 'Falha na Capabilidade': 'Não há',
 'Limites Especificados': [2.7, 3.1],
 'Média Amostral': 2.98568,
 'R&R': 0.069,
 'Repetibilidade': 0.01157,
 'Reprodutibilidade': 0}

# Narrativa

A função criada retorna valores relevantes à Análise do Sistema de Medição. Desta forma, podemos iniciar a avaliação. Supondo, portanto, que a condição inicial do sistema seja dada por:

In [ ]:
primeiraAnalise = avSisMed(n=200,dp=0.08,dpPeca=0.05,dp1=0.1)
primeiraAnalise[0]

{'%Peça a Peça': 47.82,
 '%R&R': 87.82,
 'Capacidade dos Medidores': {'op1': 1.016, 'op2': 0.511, 'op3': 0.478},
 'Desvio Padrão Peça a Peça': 0.364,
 'Desvio Padrão Total': 0.761,
 'Média Amostral': 2.99782,
 'R&R': 0.668,
 'Repetibilidade': 0.11141,
 'Reprodutibilidade': 0}

Desta primeira análise podemos observar que há de fato um problema com o nosso processo. Temos um desvio padrão total excessivamente alto ($0,96$). Além disso, percebemos que a porcentagem de R&R na medição é muito alta ($71,2 \%$).

Entretanto, um detalhe curioso é o fato da capacidade do medidor 1 ter um erro muito maior em relação aos outros dois medidores. Podemos visualizar esse dado através de um histograma sobreposto de todos os medidores:

$\bar{X}$ 


In [ ]:
df = primeiraAnalise[2]
dfprint(df)
hist = px.histogram(df,
                    x=["Op1_xBarra","Op2_xBarra","Op3_xBarra"],
                    nbins=20,
                    marginal="box",
                    title="Distribuição da Média Amostral por Operador")
hist.show()

Percebemos nas ditribuções do histograma que o Operador 1 tem uma largura superior aos demais, o que é confirmado pelo boxplot acima do histograma. Portanto, como primeira intervenção, nós podemos oferecer um WorkShop de utilização do paquímetro, para que possamos melhorar a aferição do Medidor 1.

Após a intervenção, realizamos nossa segunda análise:

In [ ]:
segundaAnalise = avSisMed(n=200,dp=0.08,dpPeca=0.05)
segundaAnalise[0]

# df = segundaAnalise[2]
# dfprint(df)
# hist = px.histogram(df,
#                     x=["Op1_xBarra","Op2_xBarra","Op3_xBarra"],
#                     nbins=20,
#                     marginal="box",
#                     title="Distribuição da Média Amostral por Operador")
# hist.show()

{'%Peça a Peça': 61.22,
 '%R&R': 79.07,
 'Capacidade dos Medidores': {'op1': 0.492, 'op2': 0.458, 'op3': 0.487},
 'Desvio Padrão Peça a Peça': 0.371,
 'Desvio Padrão Total': 0.606,
 'Média Amostral': 2.9951,
 'R&R': 0.479,
 'Repetibilidade': 0.0798,
 'Reprodutibilidade': 0}

Bom, conseguimos normalizar os medidores, reduzindo essa primeira variação. Podemos agora mitigar outros problemas.

> *Quando comparamos as medidas tomadas por um único medidor, estamos avaliando a **Repetibilidade**, já quando buscamos a comparação por entre os medidores, estamos avaliando a **Reprodutibilidade**.*

Podemos notar que, entre a primeira e segunda análise, a reprodutilibilidade foi a 0, indicando que talvez seja prudente agora avaliar a repetibilidade. Se estamos comparando então a variância entre as medidas, existem duas parcelas que causam ela: 
* a variância do processo: diz respeito ao erro associado a produção
* a variância da medição: diz respeito ao erro de tomada de medida (p. ex.: alta incerteza do aparelho, ambiente pouco iluminado, etc.)

É importante, primeiro, reduzir a variância da medição antes de atacar o processo; pois será impossível validar um processo sob controle se a forma como aferimos isso não possui a acurácia adequada. Essa hipótese é confirmada pelo alto valor de %R&R.

Levamos então a proposta de uma nova intervenção, e após uma análise detalhada, sugerimos:
* Compra de um paquímetro digital, com 2 casas decimais a mais (reduzindo, também, o erro de leitura do operador);
* Troca da iluminação da área onde são feitas as medidas;

Após essa intervenção, fazemos então uma terceira análise:

In [ ]:
terceiraAnalise = avSisMed(n=200,dp=0.005,dpPeca=0.05)
terceiraAnalise[0]

{'%Peça a Peça': 99.56,
 '%R&R': 9.38,
 'Capacidade dos Medidores': {'op1': 0.028, 'op2': 0.03, 'op3': 0.033},
 'Desvio Padrão Peça a Peça': 0.321,
 'Desvio Padrão Total': 0.323,
 'Média Amostral': 3.00281,
 'R&R': 0.03,
 'Repetibilidade': 0.00505,
 'Reprodutibilidade': 0}

Após a implementação a segunda intervenção, nós conseguimos reduzir o impacto da variação da medição na variação total do processo. Desta forma, temos um Sistema de Medição das Placas validado.

Entretanto, este é apenas um passo inical considerando o princípio do *kaizen*, agora podemos focar de verdade em melhorar o processo, sabendo que teremos a capacidade de aferir esta melhora.

In [ ]:
df = terceiraAnalise[2]
dfprint(df)
hist = px.histogram(df,
                    x=["Op1_xBarra","Op2_xBarra","Op3_xBarra"],
                    nbins=20,
                    marginal="box",
                    title="Distribuição da Média Amostral por Operador")
hist.show()